Including orderbook analysis from kucoin



In [3]:
!pip install pymc numpy pandas scikit-learn ccxt


  Using cached numpy-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-lea

In [8]:
import ccxt
import pandas as pd
import numpy as np
from sklearn.linear_model import BayesianRidge
from datetime import datetime, timedelta
import joblib

# Define global variables for prior data storage
MODEL_FILE = "bayesian_model.pkl"

def fetch_4h_data(api, asset, limit=200):
    """Fetch 4-hour OHLCV data from KuCoin."""
    try:
        ohlcv = api.fetch_ohlcv(asset, timeframe="4h", limit=limit)
        df = pd.DataFrame(ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        return df
    except Exception as e:
        print(f"Error fetching data for {asset}: {e}")
        return None

def fetch_order_book(api, asset):
    """Fetch the order book for a specific asset."""
    try:
        order_book = api.fetch_order_book(asset, limit=20)  # Limit must be 20 or 100 for KuCoin
        bid_depth = sum([x[1] for x in order_book['bids']])
        ask_depth = sum([x[1] for x in order_book['asks']])
        bid_ask_spread = abs(order_book['bids'][0][0] - order_book['asks'][0][0])
        return bid_depth, ask_depth, bid_ask_spread
    except Exception as e:
        print(f"Error fetching order book for {asset}: {e}")
        return None, None, None

def preprocess_data(df):
    """Preprocess data to compute features."""
    # Calculate OBV
    df['obv'] = (np.sign(df['close'].diff()) * df['volume']).cumsum()
    df['obv_change'] = df['obv'].pct_change()

    # Calculate RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14, min_periods=1).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14, min_periods=1).mean()
    rs = gain / (loss + 1e-10)  # Avoid division by zero
    df['rsi'] = 100 - (100 / (1 + rs))

    # Calculate ATR
    df['tr'] = np.maximum(df['high'] - df['low'],
                          np.maximum(abs(df['high'] - df['close'].shift()),
                                     abs(df['low'] - df['close'].shift())))
    df['atr'] = df['tr'].rolling(window=14, min_periods=1).mean()

    # Drop rows with NaN values
    df = df.dropna().reset_index(drop=True)
    return df
def train_bayesian_model(X, y, feature_names):
    """Train or update the Bayesian model."""
    # Load the model if it exists; otherwise, create a new one
    try:
        model = joblib.load(MODEL_FILE)
        print("Loaded existing Bayesian model.")

        # Check if the features match
        if model.n_features_in_ != len(feature_names):
            print("Feature mismatch. Retraining the model with new features.")
            model = BayesianRidge()
    except FileNotFoundError:
        model = BayesianRidge()
        print("Initialized new Bayesian model.")

    # Update the model with the new data
    model.fit(X, y)

    # Save the updated model
    joblib.dump(model, MODEL_FILE)
    return model

def fetch_signals_bayesian(api, assets):
    """Fetch signals using Bayesian regression for multiple assets."""
    results = []
    for asset in assets:
        # Fetch historical OHLCV data
        df = fetch_4h_data(api, asset)
        if df is None:
            continue

        # Fetch order book data
        bid_depth, ask_depth, bid_ask_spread = fetch_order_book(api, asset)
        if bid_depth is None or ask_depth is None:
            continue

        # Preprocess data
        df = preprocess_data(df)

        # Add order book features
        df['bid_depth'] = bid_depth
        df['ask_depth'] = ask_depth
        df['bid_ask_spread'] = bid_ask_spread

        # Define feature columns
        feature_names = ["obv_change", "rsi", "atr", "bid_depth", "ask_depth", "bid_ask_spread"]
        X = df[feature_names]
        y = df["close"].shift(-1).dropna()
        X = X.iloc[:-1]

        # Train/update the Bayesian model
        model = train_bayesian_model(X, y, feature_names)

        # Predict the next close price
        latest_data = X.iloc[-1].values.reshape(1, -1)
        predicted_price, uncertainty = predict_with_bayesian_model(model, latest_data)

        # Get the latest actual close price
        latest_close = df['close'].iloc[-1]

        # Generate signal
        signal = "Buy" if predicted_price > latest_close else "Sell"

        results.append({
            "asset": asset,
            "signal": signal,
            "current_price": latest_close,
            "predicted_price": predicted_price[0],
            "uncertainty": uncertainty[0],
            "rsi": df['rsi'].iloc[-1],
            "atr": df['atr'].iloc[-1],
            "obv_change": df['obv_change'].iloc[-1],
            "bid_depth": bid_depth,
            "ask_depth": ask_depth,
            "bid_ask_spread": bid_ask_spread
        })
    return results


def display_bayesian_signals(signals):
    """Display Bayesian signals in a user-friendly format."""
    print("\n--- Bayesian Trading Signals ---")
    for signal in signals:
        print(f"Asset: {signal['asset']}")
        print(f"   Signal: {signal['signal']} (Predicted Price > Current Price: Buy)")
        print(f"   Current Price: ${signal['current_price']:.6f}")
        print(f"   Predicted Price: ${signal['predicted_price']:.6f} ± ${signal['uncertainty']:.6f}")
        print(f"   RSI: {signal['rsi']:.2f} (Above 50: Bullish; Below 50: Bearish)")
        print(f"   ATR: {signal['atr']:.5f} (Higher indicates more volatility)")
        print(f"   OBV Change: {signal['obv_change']:.4f} (Positive = Buying Pressure; Negative = Selling Pressure)")
        print(f"   Bid Depth: {signal['bid_depth']}")
        print(f"   Ask Depth: {signal['ask_depth']}")
        print(f"   Bid-Ask Spread: ${signal['bid_ask_spread']:.5f}")
        print()

# Initialize CCXT for KuCoin
exchange = ccxt.kucoin()

# Define the list of assets
assets = ["BTC/USDT", "ETH/USDT", "ADA/USDT", "NEAR/USDT", "XRP/USDT", "DOGE/USDT"]

# Fetch Bayesian signals
signals = fetch_signals_bayesian(exchange, assets)

# Display the signals
display_bayesian_signals(signals)


Loaded existing Bayesian model.
Feature mismatch. Retraining the model with new features.


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Loaded existing Bayesian model.


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Loaded existing Bayesian model.


...

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Loaded existing Bayesian model.


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(


Loaded existing Bayesian model.

--- Bayesian Trading Signals ---
Asset: BTC/USDT
   Signal: Sell (Predicted Price > Current Price: Buy)
   Current Price: $102881.300000
   Predicted Price: $98515.895520 ± $3614.044261
   RSI: 42.06 (Above 50: Bullish; Below 50: Bearish)
   ATR: 1730.27857 (Higher indicates more volatility)
   OBV Change: 0.0272 (Positive = Buying Pressure; Negative = Selling Pressure)
   Bid Depth: 2.99543664
   Ask Depth: 1.6305407300000005
   Bid-Ask Spread: $0.10000

Asset: ETH/USDT
   Signal: Buy (Predicted Price > Current Price: Buy)
   Current Price: $3197.930000
   Predicted Price: $3305.547120 ± $211.301311
   RSI: 38.23 (Above 50: Bullish; Below 50: Bearish)
   ATR: 74.24000 (Higher indicates more volatility)
   OBV Change: -0.0200 (Positive = Buying Pressure; Negative = Selling Pressure)
   Bid Depth: 45.80697130000001
   Ask Depth: 34.9766252
   Bid-Ask Spread: $0.01000

Asset: ADA/USDT
   Signal: Sell (Predicted Price > Current Price: Buy)
   Current Price

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but BayesianRidge was fitted with feature names
  warnings.warn(
